In [106]:
import json, os
import random
import argparse
import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F


from tqdm import trange

from data import load_data_instances, DataIterator
from model import MultiInferBert
import utils
from transformers import BertTokenizer
import pdb

import sys
sys.argv

['G:\\Anaconda\\lib\\site-packages\\ipykernel_launcher.py',
 '-f',
 'C:\\Users\\User\\AppData\\Roaming\\jupyter\\runtime\\kernel-405377be-7a9f-49c8-be8d-e339a3b7c22e.json']

In [63]:
parser = argparse.ArgumentParser()

parser.add_argument('--prefix', type=str, default="../../data/",
                    help='dataset and embedding path prefix')
parser.add_argument('--model_dir', type=str, default="savemodel/",
                    help='model path prefix')
parser.add_argument('--task', type=str, default="pair", choices=["pair", "triplet"],
                    help='option: pair, triplet')
parser.add_argument('--mode', type=str, default="train", choices=["train", "test"],
                    help='option: train, test')
parser.add_argument('--dataset', type=str, default="res14", choices=["res14", "lap14", "res15", "res16"],
                    help='dataset')
parser.add_argument('--max_sequence_len', type=int, default=100,
                    help='max length of a sentence')
parser.add_argument('--device', type=str, default="cpu",
                    help='gpu or cpu')

parser.add_argument('--bert_model_path', type=str,
                    default="pretrained/bert-base-uncased",
                    help='pretrained bert model path')
parser.add_argument('--bert_tokenizer_path', type=str,
                    default="bert-base-uncased",
                    help='pretrained bert tokenizer path')
parser.add_argument('--bert_feature_dim', type=int, default=768,
                    help='dimension of pretrained bert feature')

parser.add_argument('--nhops', type=int, default=1,
                    help='inference times')
parser.add_argument('--batch_size', type=int, default=3,
                    help='bathc size')
parser.add_argument('--epochs', type=int, default=100,
                    help='training epoch number')
parser.add_argument('--class_num', type=int, default=4,
                    help='label number')

_StoreAction(option_strings=['--class_num'], dest='class_num', nargs=None, const=None, default=4, type=<class 'int'>, choices=None, help='label number', metavar=None)

In [64]:
args = parser.parse_args("")
print(args)
args.class_num = 6
args.task = "triplet"


Namespace(prefix='../../data/', model_dir='savemodel/', task='pair', mode='train', dataset='res14', max_sequence_len=200, device='cpu', bert_model_path='pretrained/bert-base-uncased', bert_tokenizer_path='bert-base-uncased', bert_feature_dim=768, nhops=1, batch_size=3, epochs=100, class_num=4)


In [82]:
tokenizer = BertTokenizer.from_pretrained(args.bert_tokenizer_path)
model_path = "./savemodel/berttriplet.pt"
model = torch.load(model_path, map_location=torch.device('cpu'), )
model.args.max_sequence_len = args.max_sequence_len
model.eval()

MultiInferBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [100]:
# This cell is where your actual TODOs start
# You will need to implement the Dataset class by your own. You may also implement it similar to HW1P2 (dont require context)
# The steps for implementation given below are how we have implemented it.
# However, you are welcomed to do it your own way if it is more comfortable or efficient. 

class AmazonReviews(torch.utils.data.Dataset):

    def __init__(self, data_path, partition= "train"): # You can use partition to specify train or dev
        
        file = data_path
        self.data = pd.read_csv(data_path).to_numpy()
       
       
    def __len__(self):
        
        return len(self.data)

    def __getitem__(self, ind):

        product_id, sentence = self.data[ind]


        return product_id, sentence
    
    def collate_fn(batch):
        
        
        
        list_of_review_dicts = []
        for product_id, sentence in batch:
            list_of_review_dicts.append({'id': product_id, 'sentence': sentence, 'triples': []})
        
        #The output here is [{'id': product_id, 'sentence': sentence, 'triples': []}, ...]
        return list_of_review_dicts


In [101]:
batch_size = 3

root = "../../data/amazon/small_dataset.csv" 

train_data = AmazonReviews(root, 'train')

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=AmazonReviews.collate_fn)# TODO: Define the train loader. Remember to pass in a parameter (function) for the collate_fn argument 

print("Batch size: ", batch_size)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))


Batch size:  3
Train dataset samples = 10, batches = 4


In [102]:

# Test code for checking shapes and return arguments of the train
for data in train_loader:
    x = data 
    
    print(x)
 
    break

[{'id': 'FOURTH_ID', 'sentence': "The grinder's initial build quality and workmanship seemed good considering the price. However, as someone new to using angle grinders, I was unaware of the risk of damaging the motor via overheating. Within 24 hours of opening the box, I had damaged the device by overworking it while grinding concrete.", 'triples': []}, {'id': 'FIFTH_ID', 'sentence': "Looked fantastic, lovely box , plenty spare disks etc. Finally got to use it cutting 20mm paving stones , lasted ten minutes - my own stupid fault - did not notice the 110V and plugged it in to 240V- I'm sure it would have been fine like the rest of my MAKITA tools - cut the stone perfectly in the ten minutes it worked, and I'm impressed it did not burn my hands off", 'triples': []}, {'id': 'TENTH_ID', 'sentence': "I got this for my boyfriend for Christmas and he has used it almost twice a day, every day since. It's very sturdy and durable. Make sure you put it on a stable surface before using. We origin

In [103]:
def get_pseudo_labels(pseudo_sentence_pack, sentence_ids, tokens, lengths, masks, sens_lens, token_ranges, aspect_tags, tags):
    
    
    
    all_ids = []
    all_preds = []
    all_labels = []
    all_lengths = []
    all_sens_lengths = []
    all_token_ranges = []

    all_preds.append(preds)
    all_labels.append(tags)
    all_lengths.append(lengths)
    all_sens_lengths.extend(sens_lens)
    all_token_ranges.extend(token_ranges)
    all_ids.extend(sentence_ids)

    all_preds = torch.cat(all_preds, dim=0).cpu().tolist()
    all_labels = torch.cat(all_labels, dim=0).cpu().tolist()
    all_lengths = torch.cat(all_lengths, dim=0).cpu().tolist()


    metric = utils.Metric(args, all_preds, all_labels, all_lengths, all_sens_lengths, all_token_ranges, ignore_index=-1)
    precision, recall, f1 = metric.score_uniontags()
    
    aspect_results = metric.score_aspect()
    opinion_results = metric.score_opinion()
    
    for i in range(len(metric.predictions)):
            
        predicted_aspect_spans = metric.get_spans(metric.predictions[i], metric.sen_lengths[i], metric.tokens_ranges[i], 1)
        predicted_opinion_spans = metric.get_spans(metric.predictions[i], metric.sen_lengths[i], metric.tokens_ranges[i], 2)
        if metric.args.task == 'pair':
            predicted_tuples = metric.find_pair(metric.predictions[i], predicted_aspect_spans, predicted_opinion_spans, metric.tokens_ranges[i])
        elif metric.args.task == 'triplet':
            predicted_tuples = metric.find_triplet(metric.predictions[i], predicted_aspect_spans, predicted_opinion_spans, metric.tokens_ranges[i])
        
        pseudo_sentence_pack[i]['triples'].extend(predicted_tuples)
        
    return pseudo_sentence_pack

In [104]:
pseudo_sentence_packs = []
for review_dicts in train_loader:
    
    unlabeled_sentence_pack = review_dicts
    
    instances = load_data_instances(unlabeled_sentence_pack, args)
   
    testset = DataIterator(instances, args)
    
    sentence_ids, tokens, lengths, masks, sens_lens, token_ranges, aspect_tags, tags = testset.get_batch(0)
   
    pred = model(tokens, masks)
   
    preds = torch.argmax(pred, dim=3)
    

   
    pseudo_labeled_pack = get_pseudo_labels(unlabeled_sentence_pack, sentence_ids, tokens, lengths, masks, sens_lens, token_ranges, aspect_tags, tags)
    
    pseudo_sentence_packs.extend(pseudo_labeled_pack)
    
    
    
    

In [105]:
pseudo_sentence_packs

[{'id': 'SIXTH_ID',
  'sentence': "Wow. That's all I can say. This is possibly the best vacuum I have ever owned. I usually buy the cheaper ones from Walmart So I don't have much to compare this to when it comes down to other highquality name brand vaccuums.",
  'triples': [[11, 11, 10, 10, 5]]},
 {'id': 'FIFTH_ID',
  'sentence': "Looked fantastic, lovely box , plenty spare disks etc. Finally got to use it cutting 20mm paving stones , lasted ten minutes - my own stupid fault - did not notice the 110V and plugged it in to 240V- I'm sure it would have been fine like the rest of my MAKITA tools - cut the stone perfectly in the ten minutes it worked, and I'm impressed it did not burn my hands off",
  'triples': [[3, 3, 1, 2, 5], [7, 7, 5, 6, 5], [16, 16, 5, 6, 5]]},
 {'id': 'TENTH_ID',
  'sentence': "I got this for my boyfriend for Christmas and he has used it almost twice a day, every day since. It's very sturdy and durable. Make sure you put it on a stable surface before using. We origin

In [122]:
index = 8
sentence = pseudo_sentence_packs[index]['sentence']
triplets = pseudo_sentence_packs[index]['triples']

encoding = tokenizer.encode(sentence)
words = tokenizer.convert_ids_to_tokens(encoding)
words = np.asarray(words)

print(words)
print(sentence)
print(triplets)


['[CLS]' 'this' 'vacuum' 'is' 'amazing' '.' 'i' 'vacuum' '##ed' 'with'
 'my' 'old' 'vacuum' ',' 'then' 'let' 'my' 'room' '##ba' 'vacuum' ','
 'and' 'immediate' 'after' 'i' 'vacuum' '##ed' 'with' 'this' 'one' '.' 'i'
 'had' '3' '/' '4' 'of' 'a' 'dirt' 'bin' 'full' 'of' 'dust' '&' 'dirt' '.'
 'my' 'carpets' 'and' 'rug' '##s' 'are' 'completely' 're' '##ju' '##ven'
 '##ated' '.' '[SEP]']
This vacuum is AMAZING. I vacuumed with my old vacuum, then let my roomba vacuum, and immediate after I vacuumed with this one. I had 3/4 of a dirt bin full of dust & dirt. My carpets and rugs are completely rejuvenated.
[[1, 1, 3, 3, 5], [38, 38, 41, 41, 5]]


In [ ]:
def show_triplets(label_sentence_pack):
    #TODO
    
    return NotImplemented
    